In [ ]:
# AiiDA imports.
%load_ext aiida
%aiida
from aiida import common, orm

In [1]:
from IPython.display import display, clear_output
import utils
import ipywidgets as ipw
from app_widgets import AppWidgets

In [62]:
VIEWERS = {
    "CP2K_AdsorptionE": "view_adsorption_energy.ipynb",
    "CP2K_GeoOpt": "view_geometry_optimization.ipynb",
    "CP2K_CellOpt": "view_geometry_optimization.ipynb",
    "CP2K_ORBITALS": "view_orbitals.ipynb",
    "CP2K_PDOS": "view_pdos.ipynb",
    "CP2K_STM": "view_stm.ipynb",
    "CP2K_AFM": "view_afm.ipynb",
    "CP2K_HRSTM": "view_hrstm.ipynb",
    "CP2K_Phonons": "view_phonons.ipynb",
    "CP2K_NEB": "view_neb.ipynb",
    "CP2K_Replica": "view_replica.ipynb",
    "ReplicaWorkChain": "view_replica.ipynb",
}

def uuids_to_nodesdict(uuids):
    workflows = {}
    nworkflows = 0
    for uuid in uuids:
        try:
            node = orm.load_node(uuid)
            nodeisobsolete = "obsolete" in node.extras and node.extras["obsolete"]
            if node.label in VIEWERS and not nodeisobsolete:
                nworkflows += 1
                if node.label in workflows:
                    workflows[node.label].append(node)
                else:
                    workflows[node.label] = [node]
        except common.NotExistent:
            pass

    return nworkflows, workflows

def get_all_structures_and_geoopts(node):
    """Get all atomistic models that led to the one used in the STM simulation"""
    current_node = node
    all_structures = [node]
    all_geoopts = []
    while current_node is not None:
        if isinstance(current_node, orm.StructureData):
            current_node = current_node.creator
        elif isinstance(current_node, orm.CalcJobNode):
            current_node = current_node.caller
            
        elif isinstance(current_node, orm.WorkChainNode):
            if "GeoOpt" in current_node.label:
                all_geoopts.append(current_node)
                current_node = current_node.inputs.structure
                all_structures.append(current_node)
            else:
                current_node = current_node.caller
    
    return all_structures, all_geoopts

def get_workflows(start_date, end_date):
    qb = orm.QueryBuilder()
    qb.append(
        orm.StructureData,
        filters={
            "extras": {"has_key": "surfaces"},
            "mtime": {"and": [{"<=": end_date}, {">": start_date}]},
        },
    )
    qb.order_by({orm.StructureData: {"mtime": "desc"}})

    # For each structure in QB create a dictionary with info on the workflows computed on it.
    data = []
    for node in qb.all(flat=True):
        # print("node ", node.pk, " extras ", node.extras["surfaces"])
        extras = node.extras["surfaces"]
        nworkflows = 0
        if isinstance(extras, list):
            nworkflows, workflows = uuids_to_nodesdict(node.extras["surfaces"])
        if nworkflows > 0:
            data.append(workflows)
    
    return data

In [2]:
# Widgets
config = utils.read_json("config.json")

# Send simulation to openBIS

In [3]:
# Initialise app widgets
app_widgets = AppWidgets("config.json")
app_widgets.load_dropdown_lists()

## Select experiment and molecule

## Select simulation

In [ ]:
import datetime as datetime
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=20)
data = get_workflows(start_date, end_date)

all_workflows = []
for workflows in data:
    workflows = workflows.values()
    workflows = list(workflows)[0]
    all_workflows.extend(workflows)

workflows_dropdown = utils.Dropdown(description='Workflows', disabled=False, layout = ipw.Layout(width = '993px'), style = {'description_width': "110px"}, options = all_workflows)
display(workflows_dropdown)

In [ ]:
def upload_simulations(b):
    # Get STM Simulation Workchain from AiiDA
    stm_workchain_node = workflows_dropdown.value
    structure_stm = stm_workchain_node.inputs.structure
    structure_stm_pk = structure_stm.pk
    structure_stm_node = load_node(structure_stm_pk)

    # Get Geometry Optimisation Workchain from AiiDA
    all_structures, all_geoopts = get_all_structures_and_geoopts(structure_stm_node)
    all_structures_exist_openbis, all_geoopts_exist_openbis, all_simulations_exist_openbis = [], [], []
    print(all_structures)
    print(all_geoopts)

In [ ]:
# display(app_widgets.object_dropdown)
display(app_widgets.save_close_buttons_hbox)
app_widgets.create_button.on_click(upload_simulations)
app_widgets.quit_button.on_click(app_widgets.close_notebook)